# TODO
- vP is not allow to be 0
- add everything to dictionaries
- attacker only works with dict
- optimise
- try catch when non invertible prime



In [154]:
MIN_PRIME = 0
MAX_PRIME = 10

msg = 12345678900987654321

In [155]:
import sympy
import random

def genPrime():
    return sympy.randprime(pow(2,MIN_PRIME), pow(2,MAX_PRIME))

def genPrimeWithLimit(max):
    return sympy.randprime(pow(2,MIN_PRIME), max)

def genSmallPrime():
    return sympy.randprime(pow(2,MIN_PRIME/2), pow(2,MAX_PRIME/2))

def fermatPrime():
    i = random.randint(0,4)
    return pow(2, pow(2, i)) + 1

def mmi(E, phiN):
    return pow(E, -1, phiN)

def calcN(P, Q):
    return P*Q

def calcPhiN(P, Q):
    return (P-1) * (Q-1)

def toBinary(dec):
    return "{0:b}".format(dec)

def toDecimal(bin):
    return int(bin, 2)

def concatenateInBinary(prev, tail):
    prev = toBinary(prev)
    tail = toBinary(tail)

    while len(prev) <= MAX_PRIME:
        prev = "0" + prev

    while len(tail) <= MAX_PRIME:
        tail = "0" + tail

    return toDecimal(prev + tail)

def splitInBinary(bin):
    bin = toBinary(bin)
    while len(bin) <= MAX_PRIME*2:
        bin = "0" + bin
    prev = bin[:len(bin)//2]
    tail = bin[len(bin)//2:]
    return(toDecimal(prev), toDecimal(tail))



# Voraussetzungen

In [156]:
a_P = genSmallPrime()
a_Q= genSmallPrime()


a_PhiN = calcPhiN(a_P, a_Q)
a_N = calcN(a_P, a_Q)

a_E = fermatPrime()
a_D = mmi(a_E, a_PhiN)

Attacker = {
    "P" : a_P,
    "Q" : a_Q,
    "PhiN" : a_PhiN,
    "N" : a_N,
    "E" : a_E,
    "D" : a_D
}

print(Attacker)

{'P': 17, 'Q': 23, 'PhiN': 352, 'N': 391, 'E': 257, 'D': 289}


# Generierung und Verschlüsselung

## Schritt 1, 2 und 3

In [157]:
r_to_large = 0

# Do I have a problem when R = 0?
# This has to be optimized
# BAD Inspiration from https://www.cryptool.org/assets/ctp/documents/Kleptografie_kes_4-2010.pdf
while True:
    # Schritt 1
    P = genPrime()
    vP = pow(P, a_E ,a_N)

    # Schritt 2
    t = genPrime()
    tempN = concatenateInBinary(vP, t)
    
    # Schritt 3
    # This is wrong, search for Q first. len(Q) ~ len(P). Q not max
    Q = int(genPrimeWithLimit(int(tempN/P)))
    R = tempN - P*Q

    if (R <= pow(2,MAX_PRIME) and R > 0):
        break
    r_to_large += 1

print("Amount of failed attempts",r_to_large)

Attacker["vP"] = vP
Attacker["R"] = R

print(R, Q)

print(vP, t)

print(tempN)

Attacker["vP"] = vP
Attacker["R"] = R

print("tempN = P * Q + R")
print(tempN, "=", P, "*", Q, "+", R)

Amount of failed attempts 244
592 379
31 397
63885
tempN = P * Q + R
63885 = 167 * 379 + 592


## Schritt 4

In [158]:
N = calcN(P, Q)

print("vP||t:", toBinary(vP), toBinary(t))
print("N':   ",toBinary(tempN))

print("N :",toBinary(N))
print("N':",toBinary(tempN))

print(N)

vP||t: 11111 110001101
N':    1111100110001101
N : 1111011100111101
N': 1111100110001101
63293


## Schritt 5

In [159]:
PhiN = calcPhiN(P, Q)
E = fermatPrime()

print(P, Q)
print(E)

D = mmi(E, PhiN)

print(PhiN, E, D)

167 379
17
62748 17 59057


In [160]:
User = {
    "P" : P,
    "Q" : Q,
    "PhiN" : PhiN,
    "N" : N,
    "E" : E,
    "D" : D
}

print(User)

{'P': 167, 'Q': 379, 'PhiN': 62748, 'N': 63293, 'E': 17, 'D': 59057}


## Schritt 6

In [161]:
cipher = pow(msg, User["E"], User["N"])
signature = pow(msg, User["D"], User["N"])

# Angriff

## Schritt 1

In [176]:
# Read the public key of user
pub_User = {
    "N" : N,
    "E" : E
}

print(pub_User)

collection_User = {
    "N" : User["N"],
    "E" : User["E"]
}

{'N': 63293, 'E': 17}


## Schritt 2

In [177]:
exp_vP = splitInBinary(User["N"])[0]

print("Known vP:",toBinary(Attacker["vP"]))
print(toBinary(User["N"]))
print(toBinary(exp_vP))

print(toBinary(Attacker["R"]))
print(Attacker["R"])




Known vP: 11111
1111011100111101
11110
1001010000
592


## Schritt 3

In [178]:
exp_P = pow(exp_vP, Attacker["D"], Attacker["N"])

exp_alt_P = pow(exp_vP+1, Attacker["D"], Attacker["N"])

print(exp_P, exp_alt_P)
print("Actual value of P:", P)

251 167
Actual value of P: 167


# Schritt 4

In [179]:
# Potential to optimize when N/P not hole Number
exp_Q = int(pub_User["N"]/exp_P)

exp_alt_Q = int(pub_User["N"]/exp_alt_P)

print(exp_Q, exp_alt_Q)
print("Actual value of Q:", User["Q"])

252 379
Actual value of Q: 379


## Schritt 5

In [180]:
exp_PhiN = int(calcPhiN(exp_P, exp_Q))

exp_alt_PhiN = int(calcPhiN(exp_alt_P, exp_alt_Q))

print(exp_PhiN, exp_alt_PhiN)
print("Actual value of PhiN:", User["PhiN"])

exp_D = mmi(pub_User["E"], exp_PhiN)

exp_alt_D = mmi(pub_User["E"], exp_alt_PhiN)

print(exp_D, exp_alt_D)
print("Actual value of D:", User["D"])

62750 62748
Actual value of PhiN: 62748
40603 59057
Actual value of D: 59057


## Schritt 6

In [181]:
mal_signature = pow(msg, exp_D, User["N"])
alt_mal_signature = pow(msg, exp_alt_D, User["N"])

if(signature == mal_signature):
    print("D is the right privat Key:",exp_D)
elif(signature == alt_mal_signature):
    print("D' is the right privat Key:",exp_alt_D)


D' is the right privat Key: 59057
